#**Aula 1 de Labortório de Aprendizado de Máquina**
#Visualização e Pré-processamento de Dados



##Imports das bibliotecas principais

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

##Análise das variáveis da base de dados (dados de classificação)

In [ ]:
#Faz a leitura do arquivo com os dados
#header = None --> o arquivo não tem cabeçalho
#names --> coloca nomes para cada coluna
data = pd.read_csv('/content/iris.data', header = None, names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']) #iris
#data = pd.read_csv('/content/abalone.data', header = None, names = ['sex', 'length', 'diameter', 'height', 'whole weight', 'shucked weight', 'viscera weight', 'shell weight', 'class']) #abalone
data.head()

In [ ]:
#Retorna as principais informações estatísticas da base de dados
data.describe()

In [ ]:
#Pode usar informações estatísticas individuais
#data.iloc --> permite selecionar valores que pertecem a uma determinada linha e coluna do conjunto de dados
#Foram selecionados somente as variáveis numéricas

iris = np.arange(0,4)
abalone = np.arange(1,8)
faixa = iris

print('Média')
print(data.iloc[:,faixa].mean())

print('\r\nVariância')
print(data.iloc[:,faixa].var())

print('\r\nMAD')
print(data.iloc[:,faixa].mad())

#Teste outras, como min, max, cov, etc.

#Teste acessar as variáveis pelo nome:
#print(data[['sepal length']].mean())
#print(data[['sepal length', 'petal width']].mean())

In [ ]:
#Duas formas diferentes de visualizar os boxplots
data.boxplot()
sb.catplot(kind = 'box', data = data)

In [ ]:
#Visualizar boxplots da variável 'sepal length' para cada uma das classes
sb.catplot(y = 'sepal length', x = 'class', kind = 'box', data = data) #iris
#sb.catplot(y = 'length', x = 'class', kind = 'box', data = data) #abalone

In [ ]:
#Visualizar gráficos pairwise (2 varíaveis por vez em um plano).
#hue coloca cor nos pontos conforme a classe que eles pertencem
#Na diagonal principal é mostrado
#kde --> estimativa da função de distribuição da variável
#hist --> histograma da variável

sb.pairplot(data, hue="class", diag_kind = 'kde')

In [ ]:
#Mostrar o mapa de calor das variáveis
sb.heatmap(data.iloc[:,faixa], cmap = 'Greens')

In [ ]:
#Visualizar mapa de calor para cada classe

#Este comando serve para pegar as classes únicas
classes = data['class'].unique()

for i in range(0,classes.size):
  data_select = data[data['class'] == classes[i]]
  fig = plt.subplots(figsize=(10,4))
  sb.heatmap(data_select.iloc[:,faixa], vmin = np.min(data_select.iloc[:,faixa].min()), vmax = np.max(data_select.iloc[:,faixa].max()))
  plt.title(classes[i])
  plt.yticks(rotation = 0)

##Aplicar PCA para visualizar os dados em 2D

In [ ]:
#Obter autovalores (eigValues) e autovetores (eigVectors)
#Compare os gráficos com matriz de covariância e matriz de coef. de correlação

#Usando matriz de covariância
eigValues, eigVectors = np.linalg.eig(data.iloc[:,faixa].cov())
print(eigValues)
print(eigVectors)

#Subtrair dos dados a média
new_data = data.iloc[:,faixa] - data.iloc[:,faixa].mean()


#Usando matriz de coef. de correlação
#eigValues, eigVectors = np.linalg.eig(data.iloc[:,faixa].corr())
#print(eigValues)
#print(eigVectors)

#Subtrair dos dados a média e divide pelo desvio padrão
#new_data = (data.iloc[:,faixa] - data.iloc[:,faixa].mean())/data.iloc[:,faixa].std()

#Seleciona os dois autovetores associados aos maiores autovalores
#M será a matriz de projeção dos dados
M = eigVectors[:,0:2]

#Projeta os dados nos autovetores principais
proj_data = new_data @ M

classes = data['class'].unique()

#Plota o gráfico
color = ['or','ob','og']
plt.figure(figsize=(20,10))
for i in range(0,classes.size):
  data_select = proj_data[data['class'] == classes[i]]
  plt.plot(data_select[1],data_select[0],color[i]) #Iris
  #plt.plot(data_select[1],data_select[0],marker='o',linestyle='',color = np.random.uniform(0,1,3)) #Abalone

##Pré-processamento de Dados

In [ ]:
#Observe a presença de NaN na base de dados (em linhas e colunas)
data = pd.read_csv('/content/AirQualityUCI.csv', sep = ';', thousands=',')
data.describe()

In [ ]:
#O comando data.fillna(-100) preenche NaN com o valor -100
data.fillna(-100)

In [ ]:
#Removendo NaN
#O comando data.dropna(axis=1,how = 'all') remove colunas que tenham somente NaN
dataNoNan = data.dropna(axis=1,how = 'all')
#O comando data.dropna(axis=0,how = 'all') remove linhas que tenham somente NaN
dataNoNan = dataNoNan.dropna(axis=0,how = 'all')
dataNoNan
#Observe que o número de linhas reduziu para 9357 e colunas para 15

In [ ]:
#Os valores iguais a -200 são valores que estão faltando.
#Podemos remover as amostras com valores faltantes
dataNoNan.replace(-200,np.nan).dropna()
#Observe que não é uma boa estratégia, pois reduz substancialmente a base de dados

In [ ]:
#Podemos preencher os valores que estão faltando com a média
dataNoNan = dataNoNan.replace(-200,dataNoNan.mean())
dataNoNan

In [ ]:
#Amostras redundantes podem ser removidas
dataNoNan = dataNoNan.drop_duplicates()
dataNoNan.shape
#Como o tamanho não reduziu, não havia amostras redundantes

In [ ]:
#Amostras redundantes podem ser removidas baseadas em um valor de variável, por exemplo, Time
dataNoNan = dataNoNan.drop_duplicates('Time')
dataNoNan

In [ ]:
#Aplicando uma padronização Z-Score para cada variável da base de dados
f = lambda x: (x - x.mean())/x.std()
dataNoNan2 = dataNoNan.iloc[:,2:-1]
sb.catplot(kind = 'box', data = dataNoNan2.apply(f))
dataNoNan2.apply(f).describe()

In [ ]:
#Aplicando uma padronização MAD para cada variável da base de dados
f = lambda x: (x - x.median())/x.mad()
dataNoNan2 = dataNoNan.iloc[:,2:-1]
sb.catplot(kind = 'box', data = dataNoNan2.apply(f))
dataNoNan2.apply(f).describe()

In [ ]:
#Aplicando uma normalização min-max para cada variável da base de dados
f = lambda x: (x - x.min())/(x.max() - x.min())
dataNoNan2 = dataNoNan.iloc[:,2:-1]
sb.catplot(kind = 'box', data = dataNoNan2.apply(f))
dataNoNan2.apply(f).describe()